In [ ]:
# import libraries
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import keras
import tensorflow_datasets as tfds

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
%matplotlib inline
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout, LSTM, Bidirectional

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_data = pd.read_csv("train-data.tsv",sep='\t', header= None)
train_data.columns =['label', 'message']
test_data = pd.read_csv("valid-data.tsv",sep='\t', header= None)
test_data.columns =['label', 'message']

In [ ]:
train_data.groupby('label').describe().T

In [ ]:
# get all the ham and spam emails
ham_msg = train_data[train_data.label =='ham']
spam_msg = train_data[train_data.label=='spam']

In [ ]:
# one of the ways to fix the data imbalance is to downsample the ham messages
ham_msg_df = ham_msg.sample(n = len(spam_msg), random_state = 44)
spam_msg_df = spam_msg
print(ham_msg_df.shape, spam_msg_df.shape)

In [ ]:
# create a dataframe with ham and spam messages
train_msg = ham_msg_df.append(spam_msg_df).reset_index(drop=True)

In [ ]:
# get length column for each text
train_msg['text_length'] = train_msg['message'].apply(len)
# calculate average length by label types
labels = train_msg.groupby('label').mean()
labels

In [ ]:
# map ham label as 0 and spam as 1
train_msg['msg_type']= train_msg['label'].map({'ham': 0, 'spam': 1})
test_data['msg_type']= test_data['label'].map({'ham': 0, 'spam': 1})

train_messages = train_msg['message']
train_answers = train_msg['msg_type']
test_messages = test_data['message']
test_answers = test_data['msg_type']

In [ ]:
# defining pre-processing hyperparameters
max_len = 50
trunc_type = "post"
padding_type = "post"
oov_tok = "<OOV>"
vocab_size = 500

In [ ]:
# use Tokenizer() to tokenize the words
tokenizer = Tokenizer(num_words = vocab_size, char_level=False, oov_token = oov_tok)
tokenizer.fit_on_texts(train_messages)

# get the word_index
word_index = tokenizer.word_index
word_index

In [ ]:
# check how many words
tot_words = len(word_index)
print('There are %s unique tokens in training data. ' % tot_words)

In [ ]:
# sequencing and padding on training and testing
training_sequences = tokenizer.texts_to_sequences(train_messages)
training_padded = pad_sequences (training_sequences, maxlen = max_len, padding = padding_type, truncating = trunc_type )
testing_sequences = tokenizer.texts_to_sequences(test_messages)
testing_padded = pad_sequences(testing_sequences, maxlen = max_len,
padding = padding_type, truncating = trunc_type)

In [ ]:
# shape of train tensor
print('Shape of training tensor: ', training_padded.shape)
print('Shape of testing tensor: ', testing_padded.shape)

In [ ]:
embeding_dim = 16
drop_value = 0.2 # dropout
n_dense = 24

# dense model architecture
model = Sequential()
model.add(Embedding(vocab_size, embeding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(24, activation='relu'))
model.add(Dropout(drop_value))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam' ,metrics=['accuracy'])

In [ ]:
# fitting a dense spam detector model
num_epochs = 32
early_stop = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(training_padded, train_answers, epochs=num_epochs, validation_data=(testing_padded, test_answers),callbacks =[early_stop], verbose=2)

In [ ]:
# model performance on test data
model.evaluate(testing_padded, test_answers)

In [ ]:
# create a dataframe
metrics = pd.DataFrame(history.history)
# rename column
metrics.rename(columns = {'loss': 'Training_Loss', 'accuracy': 'Training_Accuracy',
                         'val_loss': 'Validation_Loss', 'val_accuracy': 'Validation_Accuracy'}, inplace = True)
def plot_graphs(var1, var2, string):
    metrics[[var1, var2]].plot()
    plt.title('Dense Classifier: Training and Validation ' + string)
    plt.xlabel ('Number of epochs')
    plt.ylabel(string)
    plt.legend([var1, var2])
plot_graphs('Training_Loss', 'Validation_Loss', 'Loss')
plot_graphs('Training_Accuracy', 'Validation_Accuracy', 'Accuracy')

In [ ]:
# function to predict messages based on model
def predict_message(text):
  pred_text = []
  pred_text.append(text)
  new_seq = tokenizer.texts_to_sequences(pred_text)
  padded = pad_sequences(new_seq, maxlen = max_len, padding = padding_type, truncating = trunc_type)
  prediction = model.predict(padded)
  result = []
  for i in prediction:
    if i > 0.5:
      return (i.item(), 'spam')
    else:
      return (i.item(), 'ham')

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today only! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()